### <center>Zadanie 4</center>
#### Grupowanie na zbiorze dotyczącym marskości wątroby

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

### Wczytanie danych

In [ ]:
data = pd.read_csv('cirrhosis.csv')

status = data.Status
status_encoder = LabelEncoder()
status_encoder.fit(status)
status = status_encoder.transform(status)

data.drop(columns=['ID', 'Status'], inplace=True)
data['Stage'] = data['Stage'].astype('category')

data.info()

In [ ]:
data.head()

### Wizualizacja danych

<ol>
<li>wykresy kołowe</li>
<li>histogramy</li>
<li>macierz korelacji</li>
</ol>

### Przygotowanie danych

<ol>
<li>konwersja object do category</li>
<li>uzupełnienie danych</li>
<li>standard scaler na kolumnach float, int</li>
</ol>

#### Wykresy kołowe przedstawiają podział pacjentów według:
<ol>
<li>przyjętego lekarstwa</li>
<li>płci</li>
<li>obecności wodobrzusza</li>
<li>obecności hepatomegalii</li>
<li>obecności pajączków naczyniowych</li>
<li>obecność obrzęku w połączeniu z leczeniem diuretykami</li>
</ol>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 16))
plt.suptitle('Podział pacjentów według kategorii', fontsize=20, y=0.98)

colors = ['blue', 'yellow', 'gray', 'red', 'black']
titles = ['Przyjmowanie penicylaminy', 'Płeć', 'Wodobrzusze', 'Hepatomegalia', 'Pajączki naczyniowe', 'Obrzęk z leczeniem diuretykami', 'Stopień marskości wątroby']
object_columns = data.select_dtypes(include=['object']).columns.values
object_columns = np.append(object_columns, 'Stage')

for i, column in enumerate(object_columns):
    x, y = divmod(i, 2)
    el = data.groupby(column, dropna=False, as_index=True, observed=True).size()
    explode = [0.05 for _ in range(len(el))]
    ax[x, y].pie(
        x=el,
        explode=explode,
        labels=el.index,
        colors=colors,
        autopct='%1.1f%%',
        textprops={'fontsize': 12, 'fontweight': 'bold'},
        radius=1,
        startangle=180,
        labeldistance=1.2,
        wedgeprops={'edgecolor': 'black', 'linewidth': 2},
        normalize=True,
    )
    ax[x, y].set_title(f'{titles[i]}', fontsize=14)

ax[3, 1].axis('off')
plt.tight_layout()
plt.show()

#### Konwersja object na category + Uzupełnienie danych kategorycznych

Uzupełnienie wartości następuje losowo zachowując procenty udziału tych kategorii w liczebności bez NaN.

In [ ]:
encoders = [LabelEncoder() for _ in range(len(object_columns))]

for i, column in enumerate(object_columns):
    counts = data[column].value_counts()
    missing_count = data[column].isna().sum()
    counts *= (missing_count / counts.sum())
    counts = np.ceil(counts).astype(int)
    series = pd.Series(np.repeat(counts.index, counts))[:missing_count]
    series = series.sample(frac=1, random_state=42).reset_index(drop=True)
    data.loc[data[column].isna(), column] = series.values
    data[column] = encoders[i].fit_transform(data[column])
    data[column] = data[column].astype('category')

data.info()

#### Histogramy przedstawiają rozkład danych w kolumnach:

<ol>
<li>N_Days -> liczby dni między rejestracją, a wcześniejszym zgonem, przeszczepem lub końcem analizy badania</li>
<li>Age -> wiek pacjenta w dniach</li>
<li>Bilirubin -> stężenie bilirubiny (mg/dl)</li>
<li>Choresterol -> stężenie choresterolu (mg/dl)</li>
<li>Albumin -> albumina (mg/dl)</li>
<li>Copper -> ilość miedzi w moczu (µg/dzień)</li>
<li>Alk_Phos -> fosfataza alkaliczna (U/litr)</li>
<li>SGOT -> aminotransferaza asparaginianowa (U/ml)</li>
<li>Tryglicerides -> liczba triglicerydów</li>
<li>Platelets -> liczba płytek krwi na ml/1000</li>
<li>Prothrombin -> czas protrombinowy -> czas krzepnięcia krwi</li>
</ol>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(16, 12))

titles = ['Liczba dni między rejestracją, a końcem', 'Wiek pacjenta (dzień)', 'Stężenie bilirubiny (mg/dl)', 'Stężenie choresterolu (mg/dl)', 'Obecność albuminy (mg/dl)', 'Ilość miedzy w moczu  (µg/dzień)', 'Fosfataza alkaliczna (U/litr)', 'SGOT (U/ml)', 'Liczba triglicedydów', 'Liczba płytek krwi na ml/1000', 'Czas protrombinowy']

plt.suptitle('Rozkład wartości w kolumnach z zmiennymi liczbowymi', fontsize=20)

for i, column in enumerate(data.select_dtypes(include=np.number).columns[:-1]):
    x, y = divmod(i, 3)
    ax[x, y].hist(
        x=data[column], color='blue',
        edgecolor='black',
        linewidth=1,
        alpha=1
    )
    ax[x, y].set_title(f'Kolumna: {column}', fontsize=14)
    ax[x, y].set_ylabel('Liczebność')

ax[3, 1].axis('off')
ax[3, 2].axis('off')
plt.tight_layout()
plt.show()

#### Uzupełnienie danych numerycznych

Wykorzystanie mediany dla małych braków i interpolacji w reszcie przypadków.
Zastosowanie StandardScalera dla zminiejszenia odległości między punktami w zbiorze (ułatwi to robotę dla kMeans).

In [ ]:
number_columns = data.select_dtypes(exclude='category').columns

for i, column in enumerate(number_columns):
    if data[column].isna().sum() / len(data.index) > 0.05:
        data[column] = data[column].interpolate(method='linear', limit_direction='both')
    else:
        data[column] = data[column].fillna(data[column].median())

standardizer = StandardScaler()
after_standard = standardizer.fit_transform(data[number_columns])
data.drop(columns=number_columns, inplace=True)
data = pd.concat([data, pd.DataFrame(after_standard, columns=number_columns)], axis=1)

data.info()

In [ ]:
data.head()

#### Macierz korelacji Pearsona

In [ ]:
object_columns = data.select_dtypes(include='object').columns.values
object_columns = np.append(object_columns, 'Stage')
encoders = [LabelEncoder() for _ in range(len(object_columns))]

for i, column in enumerate(object_columns):
    data[column] = encoders[i].fit_transform(data[column])

plt.figure(figsize=(12, 10))
correlation_mat = data.corr()
plt.imshow(
    X=correlation_mat,
    cmap='Greys',
    vmax=1,
    vmin=-1,
)
plt.colorbar(label='Wartość korelacji')
plt.xticks(ticks=np.arange(len(correlation_mat.columns)), labels=correlation_mat.columns, rotation=45)
plt.yticks(ticks=np.arange(len(correlation_mat.columns)), labels=correlation_mat.columns)
plt.title('Macierz korelacji dla zbioru marskości wątroby', fontsize=14, pad=20)

for i in range(len(correlation_mat.columns)):
    for j in range(len(correlation_mat.columns)):
        text = f'{correlation_mat.iloc[i, j]:.2f}'
        if correlation_mat.iloc[i, j] >= 0.2:
            plt.text(j, i, text, ha='center', va='center', color='white', fontsize=12)

plt.grid(True)

### <center>Analiza skupień przy wykorzystaniu algorytmu KMeans, AgglomerativeClustering</center>

<b>KMeans</b> - metoda k-średnich dzieli zbiór przypadków na k skupień. Algorytm rozpoczyna działanie od losowo wybranych k środków skupień lub przypadków możliwie od siebie oddalonych. Następnie w kolejnych iteracjach przypisuje obiekty do najbliższych skupień, biorąc pod uwagę odległość od ich środków (miara euklidesowa, manhattan, czebyszewa)

<b>AgglomerativeClustering</b> - metoda hierarchiczna, która działa od dołu do góry. Na początku każdy punkt jest traktowany jako osobna grupa i potem na podstawie wybranego linkage klastry są łączone w większe.
Możliwe wartości linkage:
- Ward (połączenie Ward'a) -> metoda łącząca skupienia na zasadzie minimalizacji sumy kwadratów odległości pomiędzy członkami oraz środkiem klastra.
- Maximum/Complete linkage (kompletne łączenie klastrów) -> minimalizowanie maksymalnej odległości pomiędzy parami obserwacji klastrów.
- Average linkage (średnie połączenie klastrów) -> minimalizowanie średniej odległości pomiędzy parami obserwacji klastrów.
- Single linkage (pojedyncze łączenie klastrów) -> minimalizowanie minimalnej odległości między pomiędzy parami obserwacji klastrów.

Poniżej znajduje się metodologia testowania obu metod:
<ol>
<li>Metoda łokcia</li>
<li>Kryteria wewnętrzne</li>
<ul>
<li>Kryterium współczynnika wariancji Calińskiego-Harabasza (VCR)</li>
<li>Współczynnik zarysu (ang. Silhouette coefficient)</li>
</ul>
<li>Kryteria zewnętrzne</li>
<ul>
<li>Macierz kontyngencji</li>
<li>Indeks Randa</li>
<li>Współczynnik wzajemnej informacji (ang. Mutual Information)</li>
</ul>
</ol>

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering


fig, ax = plt.subplots(nrows=2, figsize=(12, 10))

algorithms = [KMeans(init='random', random_state=42), AgglomerativeClustering()]

for i, algorithm in enumerate(algorithms):
    wcss = []
    for j in range(1, 15):
        km = algorithm.set_params(n_clusters=j)
        km.fit(data)
        wcss.append(km.inertia_)

    ax[i].plot(
        range(1, 15),
        wcss,
        marker='o',
        markerfacecolor='red',
        markeredgecolor='black',
        markersize=5,
    )
    ax[i].set_xlabel('Liczba klastrów k', labelpad=10, fontsize=12)
    ax[i].set_ylabel('Wartość WCSS', labelpad=10, fontsize=12)
    ax[i].set_title('Metoda łokcia', pad=20, fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
the_best = KMeans(n_clusters=3, linkage='average')
the_best.fit(data)

#### Kryterium współczynnika wariancji Calińskiego-Harabasza (VCR)



In [ ]:
from sklearn.metrics import calinski_harabasz_score


calinski_harabasz_score(data, the_best.labels_)

#### Współczynnik zarysu (ang. Silhouette coefficient)

In [ ]:
from sklearn.metrics import silhouette_score


silhouette_score(data, the_best.labels_)

#### Macierz kontyngencji

In [ ]:
from sklearn.metrics.cluster import contingency_matrix


plt.figure(figsize=(12, 10))
contingency_mat = contingency_matrix(status, the_best.labels_)
plt.imshow(
    X=contingency_mat,
    cmap='plasma',
)
plt.colorbar(label='Liczba obserwacji')
plt.xlabel('Przewidziane etykiety', fontsize=12, labelpad=10)
plt.ylabel('Rzeczywiste etykiety', fontsize=12, labelpad=10)
plt.xticks(ticks=np.arange(contingency_mat.shape[0]), labels=status_encoder.classes_)
plt.yticks(ticks=np.arange(contingency_mat.shape[0]), labels=status_encoder.classes_)
plt.title('Macierz kontyngencji', fontsize=14, pad=20)

for i in range(contingency_mat.shape[0]):
    for j in range(contingency_mat.shape[1]):
        plt.text(j, i, contingency_mat[i, j], ha='center', va='center', color='white', fontsize=12)

#### Indeks Randa

Indeks Randa jest miarą służącą do ewaluacji jakości grupowania (klastrowania) poprzez porównanie dwóch różnych podziałów tego samego zbioru danych. Jest to popularna metoda oceny zgodności między algorytmicznym grupowaniem a znaną klasyfikacją referencyjną.

Główne cechy Indeksu Randa:

- Mierzy podobieństwo między dwoma grupowaniami na podstawie par obiektów
- Wartość zawiera się w przedziale [0,1], gdzie 1 oznacza całkowitą zgodność grupowań
- Uwzględnia zarówno pary obiektów przypisane do tej samej grupy, jak i pary przypisane do różnych grup

In [ ]:
from sklearn.metrics import rand_score


rand_score(status, the_best.labels_)

#### Współczynnik wzajemnej informacji (ang. Mutual Information, MI)

Zakres wartości: mi >= 0

mi = 0 -> brak relacji miedzy obiema zmiennymi

Im większy wynik tym silniejsza relacja.


#### Znormalizowany MI

Zakres wartości: 0 <= nmi <= 1

mi = 0 -> brak relacji między obiema zmiennymi
mi = 1 -> bardzo silna relacja między obiema zmiennymi

In [ ]:
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score


mi = mutual_info_score(status, the_best.labels_)
nmi = normalized_mutual_info_score(status, the_best.labels_)

(mi, nmi)